In [ ]:
import numpy as np
import pandas as pd
import re
import time
import random
import heapq
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as pltp
from matplotlib.patches import Rectangle

In [ ]:
def cifloader(file, BigRadius=50000000):
    '''
    Parse data from a .cif file, to be used in linkersetup
    Origins are to be set in Layer2, in the shape of rectangles.
    Destinations are to be set in Layer1, in the shape of circles.
    Returns two lists containing the geometry properties belonging to the destinations and origins.
    User can define the wafer radius (in nanometer), default is set to 5cm. 
    The script will disregard all other objects and layers in the cif.
    
    '''
    #initialize featurelist
    origins = []
    destinations = []

    #initialize featureId counter
    featureId = 1

    #Load as txt and parse relevant elements
    with open(file, 'r') as cif_file:
        for line in cif_file:
            line = line.strip() #remove trailing whitespace
            line = line.replace(';','') #remove semicolon

            #Store L2 into Origin geometries list (Boxes)
            if line.startswith('B'):
                line = line.split(' ')
                x_len = line[1]
                y_len = line[2]
                x_pos = line[3]
                y_pos = line[4] 
                #Parse [Type, Xposition, Yposition, Xlength, Ylength]
                origins.append(['Box',x_pos,y_pos,x_len,y_len,featureId])

            #Store L1 into Destination geometries list (Circles)
            elif line.startswith('R'):
                line = line.split(' ')
                radius = line[1]
                x_pos = line[2]
                y_pos = line[3] 
                #Parse [Type, Xlen, Ylen, X, Y]
                destinations.append(['Circle',x_pos,y_pos,radius,featureId])
            else:
                pass
            featureId += 1

    #Load the the boundary size (wafer)
    return origins, destinations

In [ ]:
origins, destinations = cifloader('some_file.cif')

In [ ]:
#Verify whether parsing has been successful by plotting the parsed geometries

fig = plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k') 
ax = fig.add_subplot() 

#set background
ax.add_patch(
    Rectangle(
        (BigRadius*-1, BigRadius*-1),
        BigRadius*2,
        BigRadius*2,
        color="k"
    ))

#set limit (wafer) background            
c = plt.Circle(
    (0, 0), 
    radius=BigRadius, 
    color='white', 
    zorder=1.5
)

plt.gca().add_artist(c)

#add destination
for circle in destinations:
    originx = int(circle[1])
    originy = int(circle[2])
    rad = int(circle[3])/2

    c = plt.Circle(
        (originx ,originy), 
        rad, 
        color='green', 
        zorder=2.5
        
    )
    ax.add_artist(c)

ax = plt.gca()

#add origins
for box in origins:
    originx = int(box[1]) - (int(box[3])/2)
    originy = int(box[2]) - (int(box[4])/2)
    ax.add_patch(
         Rectangle(
             (originx, originy),
             int(box[3]),
             int(box[4]), 
             edgecolor = 'm',
             facecolor = 'm',
             fill=True, 
             zorder=2.5
         ))
        
plt.xlabel("X-AXIS")
plt.ylabel("Y-AXIS")
plt.title("Preview")

ax.set_xlim(-1*BigRadius, BigRadius) 
ax.set_ylim(-1*BigRadius, BigRadius)
ax.set_aspect('equal')

plt.show() 

In [ ]:
def discretize((origins, destinations), gridSize=90000):
    '''
    Discretize the continuous space into a grid and create feature matrices for each parsed geometry.
    Accepts origins, destinations lists in a tuple as input and user can set the featuresize resolution changing 
    gridSize. 
    
    Returns: 
    a matrix describing the boundary
    a list of matrices describing origin features
    a list of matrices describing destination features
    
    Note that using current fabrication resolution this value should not be lower than 10000nm (10 micron).
    Smaller gridsizes means more computational expense.
    '''

    gridNumber = int((BigRadius*2)/gridSize) #number of rows and columns

    #create array that maps continuous coordinates in the x-axis
    nMat = np.linspace(-1*BigRadius, BigRadius, gridNumber, endpoint=True)
    #create matrix that describes coordinates in the x-axis, (identical along y)
    xMat = np.tile(nMat,(gridNumber,1))

    #create array that maps continous coordinates in the y-axis 
    nMat = np.flip(nMat, 0)
    yMat = nMat.reshape((-1, 1))
    yMat = np.tile(yMat,(1,gridNumber))


    #initialize grid that describes where the outer boundaries lie, (whatever is off the wafer)
    boundaryMat = np.zeros((gridNumber,gridNumber))
    #get max matrix coordinates
    xdim, ydim = xMat.shape
    
    #iterate over the matrix, changing values in boundaryMat from 0 to 1 wherever a coordinate lies outside of the boundary space
    for x_cor in range(xdim):
        for y_cor in range(ydim):
            if (float(xMat[x_cor,y_cor]))**2 + float(yMat[x_cor,y_cor])**2 > (BigRadius-(gridSize*2))**2:
                boundaryMat[x_cor, y_cor] = 1
            else:
                pass
    
    #initialize lists for origin matrices
    o_mats = []    
    #initialize lists for destination matrices
    d_mats = []
    
    #iterate over every origin feature     
    for count, origin in enumerate(origins):
        print(count,'of',len(origins))
        #initialize a matrix for that feature
        orimat = np.zeros((gridNumber,gridNumber))
        #retrieve feature parameters to fit on matrix
        xmax = float(origin[1]) + float(origin[3]) + gridSize
        xmin = float(origin[1]) - gridSize
        ymax = float(origin[2]) + float(origin[4]) + gridSize
        ymin = float(origin[2]) - gridSize
        #iterate over the matrix, changing values in orimat from 0 to 1 wherever a coordinate lies within the feature
        for x_cor in range(xdim):
            for y_cor in range(ydim):
                if xmax > float(xMat[x_cor,y_cor]) > xmin and ymax > float(yMat[x_cor,y_cor]) > ymin:
                    orimat[x_cor, y_cor] = 1
                else:
                    pass
        #add the origin matrix belonging to feature to the origin list
        o_mats.append(orimat)
        
    #iterate over every destination feature  
    for count, destination in enumerate(destinations):
        print(count,'of',len(destinations))
        #initialize a matrix for that feature
        destmat = np.zeros((gridNumber,gridNumber))
        #iterate over the matrix, changing values in destmat from 0 to 1 wherever a coordinate lies within the feature
        for x_cor in range(xdim):
            for y_cor in range(ydim):
                if (float(xMat[x_cor,y_cor]) - float(destination[1]))**2 + (float(yMat[x_cor,y_cor]) - float(destination[2]))**2 < ((float(destination[3])/2)+(2*gridSize))**2:
                    destmat[x_cor, y_cor] = 1
                else:
                    pass
        #add the destination matrix belonging to feature to the origin list
        d_mats.append(destmat)

    return boundaryMat, o_mats, d_mats, yMat, xMat


In [ ]:
boundaryMat, o_mats, d_mats, yMat, xMat = discretize((origins, destinations), gridSize=90000)

In [ ]:
#verify whether discretization worked by plotting a colormap the matrix (it's greyscale.)
#wafer will be white (0)
#outside of wafer will be lightgray (1)
#destinations (circles) will be darkgray (2)
#origins (squares) will be black (4)

#create composite matrix from the boundary matrix and update all featurematrices to appear on the matrix
composite = boundaryMat
                                                                                                                            
for dests in d_mats:
    composite[dests==1] = 2
for oris in o_mats:
    composite[oris==1] = 3
    
fig = plt.figure(figsize=(26,26), dpi= 300, facecolor='w', edgecolor='k')
ax = fig.add_subplot()

plt.imshow(composite, cmap='binary')
plt.colorbar()
plt.show()

In [ ]:
#Before proceeding, check whether the origin and destination pairs are equals and note how many combinations there are.

#Is number of Box objects equal to Circle objects?
if len(origins) == len(destinations):
    print('There are', len(origins), 'pairs.')
else: 
    print('There are', len(origins), 'origins and', len(destinations), 'destinations')

#How many permutations?
n = len(origins)
factorial = 1
if int(n) >= 1:
    for i in range (1,int(n)+1):
        factorial = factorial * i
print('Factorial of', n, 'is:', factorial, '(the number of trace configurations)')

In [ ]:
def randomize(o_mats, origins, d_mats, destinations):
    '''
    Randomize order in which origins and destinations are paired.
    Default is parsing order, or the order in which elements are placed in the .cif file.
    Returns same lists but in different order (while destinations and matrices are still paired)
    Operation is obsolete after running kNN ordering. 
    '''
    #Roll a destination (create list and then shuffle order)
    matsSh = o_mats
    originsSh = origins
    temp = list(zip(matsSh, originsSh)) 
    random.shuffle(temp) 
    o_mats, origins = zip(*temp) 

    #Roll an origin (create list and then shuffle order)
    destSh = d_mats
    destinationsSh = destinations
    temp = list(zip(destSh, destinationsSh)) 
    random.shuffle(temp) 
    d_mats, destinations = zip(*temp) 
    
    return o_mats, origins, d_mats, destinations

In [ ]:
def heuristica(h_origin, h_destination, method='octile'):
    '''
    Heuristic function returns the cost value between 
    h_origin and h_destination, which are matrix coordinates.
    Different methods of calculation can be used. 
    'octile', 'euclidean' and 'manhattan'
    Refer to the internet to figure out how it works.
    '''
    dx = abs(h_destination[0] - h_origin[0])
    dy = abs(h_destination[1] - h_origin[1])
    if method == 'octile'
        return (dx + dy) + (sqrt(2) - 2) * min(dx, dy)
    elif method == 'euclidean':
        return sqrt((dx**2) + (dy**2))
    elif method == 'manhattan':
        return (dx + dy)

In [ ]:
def astar(array, s_origin, s_destination, method='octile'):
    '''
    A* function
    Accepts a matrix containing obstacles (1) and free space (0) and an origin and destination coordinates on that matrix.
    Returns a list of tuples which represent the coordinates of the shortest path (from destination to origin).
    Different modes of movement. Use the same method as in the heuristic function.
    'octile', 'euclidean' and 'manhattan'
    Some allow diagonal and others only vertical and horizontal movement.
    '''
    if method == 'octile' or 'euclidean':
        tilesAdjacent = [(0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)] #array search area
    elif method == 'manhattan':
        tilesAdjacent = [(0,1),(0,-1),(1,0),(-1,0)] #array search area
    
    closeSet = set() #set of positions that can be left unconsidered
    
    pathsTraveled = {} #routes that have been taken. From this dictionary the final path will be retrieved.
    
    movementCost = {s_origin:0} #contains the number of steps taken each iteration
    heurScore = {s_origin:heuristica(s_origin,s_destination,method)} #contains heuristic scores (distance left)
    
    oheap = [] #Open list of paths that can be considered for shortest route
    heapq.heappush(oheap,(heurScore[s_origin],s_origin)) # prepare binary tress containing the nodes of the paths considered
                                                         # heappush() method pushes position belonging to pathstraveled into an existing heap in such 
                                                         # a way that the heap property is maintained.
    while oheap: #iterate until no positions are left
        current = heapq.heappop(oheap)[1] #read the position of smallest item from the heap
        #if current position equals the goal(destination reached)    
        if current == s_destination: 
            
            data = [] 
            
            while current in pathsTraveled:
                
                data.append(current)
                
                current = pathsTraveled[current]
                
            return data
        
        #otherwise 
        closeSet.add(current)

        #compute movementCost scores all neighbors
        for i,j in tilesAdjacent:

            adjacent = current[0] + i, current[1] + j

            currentMovementCost = movementCost[current] + heuristica(current,adjacent)

        #if adjacent is outside of the grid or bounds ignore adjacent
        
            if 0 <= adjacent[0] < array.shape[0]:
                if 0 <= adjacent[1] <array.shape[1]:
                    if array[adjacent[0],adjacent[1]] == 1:
                        #array bound internal obstacle
                        continue
                else:
                    #array bound y walls
                    continue
            else:
                #array bound x walls
                continue
                
            if adjacent in closeSet and currentMovementCost >= movementCost.get(adjacent,0):
                #if adjacent belongs to bad paths and we're already iterating worse than our best alternative
                #ignore
                continue
            
            #update queue if movementcost is lower than 
            if currentMovementCost < movementCost.get(adjacent,0) or  adjacent not in [i[1] for i in oheap]:
                pathsTraveled[adjacent] = current
                movementCost[adjacent] = currentMovementCost
                heurScore[adjacent] = currentMovementCost + heuristica(adjacent,s_destination)
                heapq.heappush(oheap,(heurScore[adjacent],adjacent))
    return False

In [ ]:
def kNNpairer(origins, destinations, yMat, xMat, method='octile')
    '''
    kNNpairer evaluates the distances between every possible combintion of origins and destinations.
    Will take the output of the prior functions and return a dictionary of destinations and a list of origins.
    The origin list is ordered from shortest distance to longest and its elements serve as key to the destination dictionary.
    The dictionary returns an list containing destinations ordered as closest to the key origin. 
    '''
    
    #pull origins coordinates
    oriSh = origins 
    destSh = destinations

    #initialize dictionary containing
    ori_dest_match_dict = {}

    #loop through origin COORDS
    for indexo, ori in enumerate(oriSh):

        #collect origin coordinates
        startCoX, startCoY = int(ori[1]) + int(ori[3])/2, int(ori[2]) + int(ori[4])/2

        #start index on array
        ori_x = (np.abs(xMat[0]-startCoX)).argmin()
        ori_y = (np.abs(yMat[:,0]-startCoY)).argmin()

        #initialize list with destinations, deltas wrt a given origin plus, plus matrix
        destList = []

        #loop through destination COORDS
        for indexD, dest in enumerate(destSh): # also loop through dests

            #collect destination coordinates
            endCoX, endCoY = int(dest[1]), int(dest[2])

            #end index on array
            dest_x = (np.abs(xMat[0]-endCoX)).argmin()
            dest_y = (np.abs(yMat[:,0]-endCoY)).argmin()

            #compute distance between pair
            dx = dest_x - ori_x
            dy = dest_y - ori_y
            
            #method dependent distances
            if method == 'octile'
                delta_Mag = (dx + dy) + (sqrt(2) - 2) * min(dx, dy)
            elif method == 'euclidean':
                delta_Mag = math.sqrt((dx**2) + (dy**2))
            elif method == 'manhattan':
                delta_Mag = (dx + dy)
            
            #add pair to list pairs
            destList.append([dest, delta_Mag, indexo, indexD, (dest_y, dest_x), (ori_y, ori_x)])
            #add distance to list distances

        #sort destinations based on shortest to longest distance wrt to current iteration of origins
        destList.sort(key = lambda x : x[1])
        #save sorted destlist with ori COORDS as key
        ori_dest_match_dict[ori[5]] = destList

    #initialize list with origins                         
    oriList = []

    #loop through dictionaries                         
    for key in ori_dest_match_dict:
        #add origin COORDS as key plus the shortest distance to any dest plus origin matrix
        oriList.append([key, ori_dest_match_dict[key][0][1]])

    #sort by shortest distance
    oriList.sort(key = lambda x : x[1])
    
    return ori_dest_match_dict, oriList

In [ ]:
ori_dest_match_dict, oriList = kNNpairer(origins, destinations, yMat, xMat, method='octile')

In [ ]:
def tracer(boundaryMat, o_mats, d_mats, yMat, xMat, oriList, ori_dest_match_dict, method='octile')
    '''
    run a* on entire dataset as prepared earlier
    
    
    Stores paths in a .txt file called newlines.txt that can be read as .cif file. 
    Shows a plot of the resulting traces.
    Potentially throws an error when the algorithm boxes itself in.
    '''
    #initialize baselayer to iteratively build obstacles
    maskLayer = boundaryMat
    #pull matrices containing obstacle or origin/destination info
    matsm = o_mats 
    dests = d_mats

    #initialize list in which linecoordinates are appended
    linecoords = []
    #initialize list with completed traces                         
    routes = []
    #initialize list with processed destinations
    processed_dests = []

    #save number of destinations that have already been linked up
    tbp = len(processed_dests)
    
    #start tracking time
    t = time.time()

    #loop through the sorted list of origins. Start with shortest pair.                         
    for it, key in enumerate(oriList):

        #set index to 0, iterate through integers which represent the next destination in the destList
        index = 0
        bottomlayer = maskLayer

        #in while loop as long as no new destination has been processed into a trace
        while tbp == len(processed_dests):

            #if the nearest destination coordinate is already occupied (thus yet linked and in the processed_dests list)
            if ori_dest_match_dict[key[0]][index][3] in processed_dests:
                #move on to the next nearest in the list
                index += 1
            else:
                #count time
                start = time.time()
                #construct a matrix where the non-origin and non-destination objects become obstacles
                #orima and destma represent the ith elements in the mats/dests matrix lists
                orima = ori_dest_match_dict[key[0]][index][2] 
                destma = ori_dest_match_dict[key[0]][index][3]
                
                #iterate through all i's
                for i in range(len(oriList)):
                    #if the i's are the ones to be linked
                    if i == orima or i == destma:
                        #we assign this as free space (2 is no obstacle)
                        bottomlayer[matsm[i]==1] = 2
                        bottomlayer[dests[i]==1] = 2
                    else:
                        #becomes an obstacle (1)
                        bottomlayer[matsm[i]==1] = 1
                        bottomlayer[dests[i]==1] = 1

                #find route in matrix, assign origin and destination matrix coordinates to variables. 
                s_origin = ori_dest_match_dict[key[0]][index][5]
                s_destination = ori_dest_match_dict[key[0]][index][4]
                print('Origin:', s_origin, '. Destination:', s_destination)

                #run astar algorithm to get the route between these
                route = astar(bottomlayer, s_origin, s_destination)

                #print('Route descriptors:', route)
                processed_dests.append(ori_dest_match_dict[key[0]][index][3])

                #if no solution can be found when running the algorithm, stop the function.
                if type(route) == type(False):
                    print('Throws error on iteration: %d. Can\'t find any solution for this iteration. It may be boxed in.', %(it)
                    break

                else:
                    #append the actual origin to the list and revert the order so it runs from origin to destination
                    route = route + [ori_dest_match_dict[key[0]][index][5]]
                    route = route[::-1]

                    #extract x and y coordinates from route list
                    x_coords = []
                    y_coords = []
                    
                    #create an new string for the .cif file describing this route
                    routeline = str(it) + ' W 50000' # let this depend on nearest line

                    # turn matrix coordinates back into continuous space coordinates
                    for i in (range(0,len(route))):
                        x = route[i][0]
                        y = route[i][1]
                        x_coords.append(x)
                        y_coords.append(y)

                        #turn x and y matrix values back into into continuous coordinate space values
                        xcor = xMat[y,x]
                        ycor = yMat[y,x]
                        
                        #add coordinates to routestring
                        routeline = routeline + ' ' + str(int(xcor)) + ' ' + str(int(ycor))
                    
                    #add newline to end of string
                    routeline =  routeline + ';\n'
                    
                    #store the path in a txt file which can be converted into a .cif.
                    file = open("newlines.txt","a") 
                    file.write(routeline) 
                    file.close() 
                    
                    #create empty matrix again
                    linearray = np.zeros((gridNumber,gridNumber))
                    
                    #store traced route into the linearray
                    for coord in route:
                        #take one coord and set its neighboring coords to 1
                        linearray[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1] = 1
                    
                    #append it to the linecoords list so all lines can be reconstructed later
                    linecoords.append([x_coords, y_coords])
                    #build the maskLayer matrix using the linearray so the currently drawnline becomes an obstacle next iteration
                    maskLayer[linearray==1] = 1

                print('completed iteration:', it, 'in', time.time() - start, 'seconds')


        #processed destinations length should increase by one, increasing loop count
        tbp = len(processed_dests)

    #overlay every path matrix onto the matrix with existing destinations, origins and boundaries.
    for i in range(len(destSh)):
        maskLayer[matsSh[i]==1] = 1
        maskLayer[destSh[i]==1] = 1

    elapsed = time.time() - t
    print('took', elapsed, 'seconds')
                          
    #plot a map of the result
    fig, ax = plt.subplots(figsize=(20,20))
    ax = fig.add_subplot()                      
    ax.imshow(maskLayer, cmap='binary')
    plt.show() 

In [ ]:
tracer(boundaryMat, o_mats, d_mats, yMat, xMat, oriList, ori_dest_match_dict, method='octile')